In [1]:
using Plots

# Le B-Spline

Una B-Spline è la rappresentazione di una curva (o meglio di una funzione) a partire dall'interpolazione degli elementi di una base normalizzata dello spazio delle funzioni analitiche.

L'interpolazione ha luogo a partire da una serie di punti del piano detti nodi (o _knots_) i quali giacciono all'interno di un poligono di controllo delimitato da una famiglia di punti $B$.

Il vettore di posizione (_i.e._ la funzione parametrica) di una B-Spline è dunque definito come:
$$
    P(t) = \sum_{i=1}^{n+1}B_iN_{i,k}(t), \qquad 2 \leq k \leq n+1
$$
dove:
 - `B_i` indica l'iesimo punto del poligono di controllo ($n+1$ punti)
 - `N_{i,k}` indica la funzione della base dell'$i$-esima B-Spline normalizzata di ordine $k$ (ovvero grado $k-1$).

In particolare un metodo per calcolare tale base `N_{i, k}` è data dalla formula ricorsiva di _Cox-de Boor_.
$$
    N_{i,k}(t) = \frac{(t-x_i)N_{i,k-1}(t)}{x_{i+k-1}-x_i} + \frac{(x_{i+k}-t)N_{i+1,k-1}(t)}{x_{i+k}-x_{i+1}},
    \qquad \mbox{con} \quad
    N_{i,1}(t) = 
    \begin{cases}
        1  &  \mbox{ se } x_i < t < x_{i+1}\\
        0  &  \mbox{ altrimenti}
    \end{cases}
$$
Ricordiamo che l'interpolazione effettuata dipende dai seguenti parametri:
 - Ogni B-Spline deve passare per $k$ nodi (o _knots_) $x$ (fatta eccezione per quelle di bordo)
 - $P(t)$ e tutte le sue derivate fino alla $k-2$-esima devono essere continue sull'intera curva (in particolare ogni B-Spline deve godere di buon incollamento con le altre) 
 - I nodi $x_i$ devono essere monotoni non decrescenti (_i.e._ $x_i \leq x_{i+1}$)
 - $P(t)$ deve essere un polinomio di grado $k-1$ su ogni intervallo $x_i \leq t < x_{i+1}$
 
In base a queste quattro caratteristiche è possibile determinare (a patto di avere sufficienti knots) delle B-spline di grado arbitrariamente alto.

## Proprietà delle B-Spline

Fra le proprietà delle curve troviamo:
 - La somma delle funzioni di base è costante per ogni valore di $t$
 $$
     \sum_{i=1}^{n+1} N_{i,k} = 1
 $$
 - Le funzioni di base sono non negative
 - L'ordine massimo della curva è al più pari al numero $n+1$ di punti del poligono di controllo (da cui il grado massimo è al più $n$)
 - Una curva può essere modificata da una funzione $f$ affine mediante l'applicazione di $f$ ai punti del poligono di controllo
 - La curva giace nello scafo connesso del suo poligono di controllo.

L'ultima affermazione ci permette di scegliere i punti del poligono di controllo dei quarti di circonferenza come:
$$
    \begin{split}
        B_I     =& [(+2, 0), (+2,+2), ( 0,+2)]\\
        B_{II}  =& [( 0,+2), (-2,+2), (-2, 0)]\\
        B_{III} =& [(-2, 0), (-2,-2), ( 0,-2)]\\
        B_{IV}  =& [( 0,-2), (+2,-2), (+2, 0)]\\
        \hline\hline
        B       =& [(+2, 0), (+2,+2), ( 0,+2),\\
                 &\ (-2,+2), (-2, 0), (-2,-2),\\
                 &\ ( 0,-2), (+2,-2), (+2, 0)]
    \end{split}
$$
ed afferma inoltre che una buona scelta per $k$ è $3$ dal momento che in tale maniera la B-Spline risultante è vincolata a passare per i punti $[(+2,0), (0,+2), (-2,0), (0,-2)]$.

Qualora si volesse calcolare una B-Spline di ordine 4 sarà invece necessario duplicare i punti indicati almeno una volta.

## I Nodi (_Knot_)

La scelta dei nodi è di cardinale importanza.

In generale se la funzione è di ordine $k$ e la costruzione avviene su un poligono di $n+1$ vertici, il numero di nodi $m$ segue la relazione:
$$
    m = k + n +1
$$

In particolare si usano due tipologie di nodi:
 - **periodici**: Presentano il primo e l'ultimo valore in molteplicità $k$
 - **aperti**: Presentano tutti i valori con molteplicità non fissata (perlopiù 1)

che si presentano in due costruzioni diverse:
 - **uniformi**: ovvero l'intervallo è suddiviso in parti uguali dai nodi
 - **non uniformi**: ovvero l'intervallo fra un nodo ed il successivo può variare (o anche essere zero)

In particolare si hanno quindi 4 classi di nodi:
 - **Periodici Uniformi**: Conducono ad una forma della funzione della base
   $$
       N_{i,k}(t) = N_{i-1,k}(t-1) = N_{i+1,k}(t+1)
   $$
 - **Aperti Uniformi**: presentano uno spaziamento uniforme interno e una molteplicità $k$ sui nodi esterni, ad esempio:
   $$
       k = 3 \qquad [0\ 0\ 0\ 1\ 2\ 3\ 3\ 3]
   $$
   Un caso particolare di questa struttura si ha quando il numero di punti del poligono di controllo è pari al grado della B-Spline: in questo caso non si ottiene altro che una base di Berntein e, conseguentemente, una curva di Bézier.
 - **Aperti Non Uniformi**
 - **Periodici Non Uniformi**

## Funzioni delle Basi per le B-Spline

Dal momento che le funzioni `N_{i,k}` definite a partire dalla formula di _Cox-de Boor_ sono in forma ricorsiva su $k$, il calcolo di un set di basi può essere ottimizzato salvando le computazioni necessarie al calcolo di più d'una fra le $n+1$ basi. A questo proposito è bene avere in mente un albero delle dipendenze:
$$
    \begin{matrix}
        N_{i, k}\\
        N_{i, k-1} & N_{i+1, k-1}\\
        N_{i, k-2} & N_{i+1, k-2} & N{i+2, k-2}\\
        \vdots & \vdots & \vdots & \ddots\\
        N_{i, 1} & N_{i+1, 1} & N{i+2, k-2} & \dots & N{i+k-1, 1}
    \end{matrix}
    \begin{matrix}
        N_{i-k+1, k} & \dots & N_{i-1, k} & N_{i, k} & N_{i+1, k} & \dots & N_{i+k-1, k}\\
                    & \ddots & \vdots     & \vdots   & \vdots     &       &             \\
                     &       & N_{i-1, 2} & N_{i, 2} & N_{i+1, 2} \\
                     &       &            & N_{i, 1} 
    \end{matrix}
$$

Ne segue che per modificare una B-Spline si possono utilizzare i seguenti artifici:
 - Cambiare il tipo di nodi (e dunque anche la base associata): aperti, periodici, uniformi, non uniformi
 - Modificare l'ordine $k$ della funzione (e dunque il grado $k-1$)
 - Variare la posizione dei punti di controllo
 - Utilizzare un numero maggiore di punti di controllo
 - Ingrandire il numero di nodi

In [2]:
"""
    knot(n, c)
Generate a `k` degreee B-spline open uniform knot vector over `n` control vertices.

An open uniform knot vector has multiplicity of knot values
 at the ends equals to the order `k` of the B-Spline basis function.
Internal knot values are evenly spaced.
The resulting open uniform basis functions yield curves
that behave most nearly like Bézier curves.

# Examples
```jldoctest
julia> knot(5, 2)
7-element Array{Int64,1}:
 0
 0
 1
 2
 3
 4
 4
```
"""

function knot(n::Int64,c::Int64)::Array{Int64}
    x=Int64[];
    nplusc::Int64 = n+c;
    nplus2::Int64 = n+2;
    push!(x, 0);
    for i = 2:nplusc
        if i>c && i<nplus2
            push!(x, last(x)+1);
        else
            push!(x, last(x));
        end
    end
    return x
end

knot

In [68]:
"""
    oknot(n1, k, centre=false, step=1.0)
Generate a `k` degree B-spline open uniform knot vector over `n1 = n+1` control vertices.

An open uniform knot vector has multiplicity of knot values
 at the ends equals to the order `k` of the B-Spline basis function.
Internal knot values are evenly spaced (uniform).
The resulting open uniform basis functions yield curves
 that behave most nearly like Bézier curves.
`centre` Boolean value indicates if the knots are centered in the
 zero value or if them goes from zero to `m-1` where `m=n1+k`.
 If `true` is choosen, then `m` must be odd.
Necessary condition is that `n1>=k` in order to correct build the knot vector.

# Examples
```jldoctest
julia> knot(5, 2)
7-element Array{Float64,1}:
 0.0
 0.0
 1.0
 2.0
 3.0
 4.0
 4.0
```

```jldoctest
julia> knot(6, 3, false, 0.25)
9-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.25
 0.5
 0.75
 1.0
 1.0
 1.0
```

```jldoctest
julia> knot(5, 2, true)
7-element Array{Float64,1}:
 -2.0
 -2.0
 -1.0
 0.0
 1.0
 2.0
 2.0
```
"""

function oknot(n1::Int64, k::Int64, centre=false::Bool, step=1.0::Float64)::Array{Float64}
    local x = Float64[];
    local m::Int64 = n1+k;
    local backstep::Float64 = 0.0;
    if centre
        if m%2!=1
            print(Error)
            return x
        else
            backstep = step*(n1-k+1)/2
        end
    end
    if n1<k
        print("Attention, k > n1")
    end
    push!(x, -backstep)
    for i = 2:m
        if i>k && i<=n1+1
            push!(x, last(x)+step)
        else
            push!(x, last(x))
        end
    end
    return x
end

oknot

In [69]:
oknot(6,3, true)

9-element Array{Float64,1}:
 -2.0
 -2.0
 -2.0
 -1.0
  0.0
  1.0
  2.0
  2.0
  2.0

In [3]:
"""
    knotu(n, c)
Generate a B-spline periodic uniform knot vector over `n` control vertices.

In a periodic uniform knot vector the knot values are evenly spaced.
Moreover each base is a translate of the others as it could be defined like:
```tex
N_{i, k}(t) = N_{i-1, k}(t-1) = N_{i+1, k}(t+1)
```

```jldoctest
julia> knotu(5, 2)
7-element Array{Int64,1}:
 0
 1
 2
 3
 4
 5
 6
```
"""

function knotu(n::Int64, c::Int64)::Array{Int64}
    x=Int64[];
    nplusc=n+c;
    nplus2=n+2;
    for i=1:nplusc
        push!(x, i-1);
    end
    return x;
end

knotu

In [4]:
B = [0.0 0.0; 0.0 2.0; 2.0 2.0]::Array{Float64, 2};
n = length(B[:,1])::Int64 -1;
k = 3::Int64;
kvec = convert(Array{Float64}, knot(n, k))::Array{Float64};

----
# Tests
----

In [17]:
function base(i::Int64, k::Int64, t::Float64, x::Array{Float64})
    if k==1
        if t>x[i] && t<x[i+1]
            return 1
        else
            return 0
        end
    else
        return ((t-x[i])*base(i, k-1, t, x))/(x[i+k-1]-x[i]) + ((x[i+k]-t)*base(i+1,k-1,t,x))/(x[i+k]-x[i+1])
    end
end

base (generic function with 1 method)

In [6]:
function position(t::Float64, B::Array{Float64, 2}, x::Array{Float64})
    local acc = 0.0;
    local n = length(B[:,1])::Int64
    for i = 1 : n+1
        for k = 2 : n+1
            acc = acc + B[i, 1] + base(i,k,t,x)
        end
    end
end

position (generic function with 1 method)

In [7]:
miabase(t) = base()

f (generic function with 1 method)

In [8]:
plot(f, 0.0, 2.0)

LoadError: [91mBoundsError: attempt to access 5-element Array{Float64,1} at index [6][39m

In [11]:
b(t) = basis(k, t, n, kvec)

b (generic function with 1 method)

In [15]:
function basis(c::Int64, t::Float64, npts::Int64, x::Array{Float64})::Array{Float64}
    temp = Array{Float64}(36);
    n = Float64[];
    nplusc = npts + c;
    
#    print(knot vector is );
#    for i=1:nplusc
#        print(& &, i, x[i]);
#    end
#    print(t is &, t);
    
    # calculate the first order basis functions n[i][1]
    
    for i=1:nplusc-1
        if (t>=x[i])&&(t<x[i+1])
            temp[i] = 1;
        else
            temp[i] = 0;
        end
    end
    
    # calculate the higher order basis functions
    for k=2:c
        for i=1:nplusc-k
            if temp[i] != 0    # if the lower order basis function is zero skip the calculation
                d = ((t-x[i])*temp[i])/(x[i+k-1]-x[i]);
            else
                d = 0;
            end
            if temp[i+1] != 0    # if the lower order basis function is zero skip the calculation
                e = ((x[i+k]-t)*temp[i+1])/(x[i+k]-x[i+1]);
            else
                e = 0;
            end
            temp[i] = d+e;
        end
    end
    
    if t == x[nplusc]    # pick the last point
        temp[npts] = 1;
    end
    
    # put in n array
    for i=1:npts
        push!(n, temp[i]);
    end
    return n;
end

basis (generic function with 2 methods)

In [16]:
b(1.0)

2-element Array{Float64,1}:
 0.0
 0.0

In [18]:
?plot

search: plot plot! plotly plot3d Plots plot3d! plotlyjs plotattr plotarea



The main plot command. Use `plot` to create a new plot object, and `plot!` to add to an existing one:

```
    plot(args...; kw...)                  # creates a new plot window, and sets it to be the current
    plot!(args...; kw...)                 # adds to the `current`
    plot!(plotobj, args...; kw...)        # adds to the plot `plotobj`
```

There are lots of ways to pass in data, and lots of keyword arguments... just try it and it will likely work as expected. When you pass in matrices, it splits by columns. To see the list of available attributes, use the `plotattr([attr])` function, where `attr` is the symbol `:Series:`, `:Subplot:`, `:Plot` or `:Axis`. Pass any attribute to `plotattr` as a String to look up its docstring; e.g. `plotattr("seriestype")`.


In [28]:
i = 0.0;
while i < 1 
   print(basis(2, i, 3, knotVec))
    i += 0.1
end

[0.0, 1.0, 0.0][0.0, 0.9, 0.1][0.0, 0.8, 0.2][0.0, 0.7, 0.3][0.0, 0.6, 0.4][0.0, 0.5, 0.5][0.0, 0.4, 0.6][0.0, 0.3, 0.7][0.0, 0.2, 0.8][0.0, 0.1, 0.9][0.0, 1.11022e-16, 1.0]

In [34]:
typeof(acc)

DataType

In [18]:
typeof(b)

#b

In [19]:
a = [b b b]

1×3 Array{#b,2}:
 b  b  b

In [21]:
a[1](1.0)

2-element Array{Float64,1}:
 0.0
 0.0

In [26]:
function mybool(flag=true::Bool)
    print(flag)
end

mybool (generic function with 2 methods)

In [29]:
mybool()

true

In [31]:
oknot(5,2)

6-element Array{Float64,1}:
 0.0
 0.0
 1.0
 2.0
 3.0
 3.0

In [33]:
for i=1:2
    print(i)
end

12